# Класс для создания сетей глубокого обучения

## **Содержание:**

1. [Использование класса](#section1)
2. [Установка и импорт необходимых пакетов](#section2)
3. [Создание класса (нейросети) с заданными параметрами](#section3)
4. [Создание и обучение модели](#section4)
5. [Частичное обучение](#section5)

<a id="section1"></a>

## 1. Использование класса

Перед использованием класса нужно **установить** следующие пакеты:
1. TensorFlow (https://www.tensorflow.org/install/)
2. Keras (https://keras.io/#installation)
2. h5py (https://pypi.org/project/h5py/)
   
`TensorFlow` – рекомендуемый бэкэнд для обучения сетей, однако возможно использовать Theano и CNTK.

`Keras` - высокоуровневая абстрактная надстройка над разными бэкэндами.

Пакет `h5py` нужен для сохранения модели на диск с последующим её извлечением, об этом подробнее далее в разделе с подробным описанием работы нейросети.

Для использования в своём коде, нужно импортировать класс **`Keras_MLP`** из файла с говорящим названием `keras_mlp.py`:

In [ ]:
from keras_mlp import Keras_MLP

Затем нужно создать экземпляр данного класса, передав ему все необходимые параметры. Список возможных параметров:

- hidden_layer_sizes
  - Это кортеж целых чисел, длина которого – количество слоёв всего, а значение при каждом индексе – количество нейронов в данном слое. Первый слой – входной. Его размерность, по идее, должна совпадать с размерностью входных данных. Последний слой – выходной, его размерность всегда должна совпадать с размерностью `y_train`.
- activations
  - Для каждого слоя нужно указать функцию активации из [всех доступных в библиотеке Keras](https://keras.io/activations/).
- alpha
  - float. Указывает величину параметра L2, предотвращающего оверфиттинг.
- batch_size
  - Указывает сколько тренировочных объектов будет показано нейросети до того, как произойдет изменение весов (можно передать int или строку `auto`, `auto` = 200)
- learning_rate_init 
  - double. Указывает скорость обучения нейросети, контролирует размер шага при изменении весов.
- max_iter # epochs
  - int. Указывает, сколько раз оптимизатор будет изменять веса.
- shuffle
  - boolean. Указывает, перемешивать ли образцы при каждом прогоне нейросети
- loss_function
  - string. Указывает [функцию потерь](https://keras.io/losses/) (необходимо для компиляции нейросети).
- metrics
  - string or list of strings. Указывает [функцию](https://keras.io/metrics/), оценивающую качество нейросети.
- verbose
  - 1/0. Выводить или нет информацию о каждом прогоне нейросети в консоль.
- early_stopping
  - boolean. Прекращать обучение или нет, если качество предсказаний не улучшается.
- optimizer_name
  - string. Название выбранного оптимизатора весов из [стандартной библиотеки `Keras`](https://keras.io/optimizers/)
- далее следуют любые параметры для выбранного оптимизатора

In [ ]:
clf = Keras_MLP(hidden_layer_sizes=(10, 10, 10, 42,),
                activations = ['relu', 'relu', 'relu', 'softmax'],
                alpha=0.00001*(2**1),
                batch_size=200, # batch_size
                learning_rate_init=0.001,
                max_iter=50, # epochs - unambigous!
                shuffle=True,
                loss_function = "categorical_crossentropy",
                metrics = ['binary_accuracy'],
                verbose=1,
                early_stopping=False,
                optimizer_name="adam",
                lr=0.001,
                beta_1 = 0.9,
                beta_2 = 0.999,
                epsilon=1e-08)

Структура дальнейшего использования такова:

Clf – это **экземпляр** класса. У него есть **метод** **`fit`**, который принимает тренировочные данные и возвращает **натренированную модель**. Эту модель затем можно **сохранить** (`model.save()`). Также у этой **модели** есть метод **`predict`**, который принимает тестовые данные и возвращает то, что предсказала. 

В общем виде структуру можно рассмотреть на приведенной диаграмме.

![caption](keras_diagram.svg)

In [ ]:
model = clf.fit(x_train, y_train)

# сохраняем модель в файл
model.save("any_colour_you_like.h5")

# предсказываем
predicted_y = model.predict(x_test)

# *Under the hood*

<a id="section2"></a>

## 2. Установка и импорт необходимых пакетов

Перед использованием класса нужно **установить** следующие пакеты:
1. TensorFlow (https://www.tensorflow.org/install/)
2. Keras (https://keras.io/#installation)
2. h5py (https://pypi.org/project/h5py/)
   
`TensorFlow` – рекомендуемый бэкэнд для обучения сетей, однако возможно использовать Theano и CNTK

`Keras` - высокоуровневая абстрактная надстройка над разными бэкэндами.

Пакет `h5py` нужен для сохранения модели на диск с последующим её извлечением, об этом подробнее далее

После установки необходимо импортировать требуемые пакеты.

- Всё, что импортируется из **`keras`** нужно для создания модели с заданными параметрами.
- **`pickle`** импортируется для извлечения законсервированных данных из `.pickle`-файлов.
- **`get_optimizer`** – это функция выбора оптимизатора весов нейросети из стандартной библиотеки `Keras`, вынесенная в отдельный файл, чтобы визуально не загружать основной код
- Импортировать **`os.path`** нужно чтобы впоследствии проверить, есть ли на диске в рабочей папке сохранённый файл нейросети.
- **`numpy`** импортируется для того, чтобы узнать размерность входных данных и настроить количество нейронов в первом и последнем уровне сети

In [ ]:
from keras import optimizers, regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
import pickle
from get_optimizer import get_optimizer
import os.path
import numpy as np

<a id="section3"></a>

## 3. Создание класса с заданными параметрами

В целом, класс состоит из трёх функций:

In [ ]:
class Keras_MLP():
    
    def __init__():
        pass

    def fit():
        pass
    
    def partial_fit():
        pass

**`def __init__():`**

Этот метод создаёт объект класса с заданными параметрами – ничего особенного. 

Правда, интересные вещи он делает в конце. В какой-то момент нейросети будет нужно задать оптимизатор весов. Однако каждый оптимизатор имеет собственные параметры; например, для того, чтобы нейросеть использовала `Adam`, ей нужно сообщить параметры `lr`, `beta_1`, `beta_2`, `epsilon` и `decay`. Однако **не всем оптимизаторам нужны именно эти параметры**, или нужны не все из них. Для того, чтобы не ограничивать себя одним оптимизатором и иметь потом возможность использовать разные их виды, сделал так, чтобы параметры именно для оптимизатора передавались в класс последними, и пары ключ-значение создавались на основании переданных значений.

In [ ]:
def __init__(self, 
             hidden_layer_sizes, 
             activations, 
             alpha, 
             batch_size, 
             learning_rate_init, 
             max_iter, # поменять на epochs? А то непонятно, что имеется ввиду
             shuffle,
             loss_function, 
             metrics, 
             verbose, 
             early_stopping,
             optimizer_name,
             **kwargs):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activations = activations
        self.alpha = alpha
        self.batch_size = batch_size
        self.learning_rate_init = learning_rate_init
        self.max_iter = max_iter
        self.shuffle = shuffle
        self.loss_function = loss_function
        self.metrics = metrics # = = = = = = = = = = = = = = = = = = = = = = = =
        self.verbose = verbose
        self.early_stopping = early_stopping
        self.optimizer_name = optimizer_name
        for key, value in kwargs.items():
          setattr(self, key, value)

Это удобно при выборе оптимизатора. Так, когда объект класса окончательно создан, в файл `get_optimizers` передаётся словарь параметров оптимизатора, откуда затем они вынимаются и присваиваются объекту выбранного оптимизатора. Этот объект затем возвращается. В том файле происходит примерно следующее:

In [ ]:
from keras import optimizers

def get_optimizer(optimizer_name, kwargs):
    if name == "adam":
		return optimizers.Adam(lr = kwargs.get('lr', 0.001),
							   beta_1 = kwargs.get('beta_1', 0.9),
							   beta_2 = kwargs.get('beta_2', 0.999),
							   epsilon = kwargs.get('epsilon', 0),
							   decay = kwargs.get('decay', 0.0))

То есть, в файл передаётся название требуемого оптимизатора и набор параметров для него. Затем он выбирается и возвращается в переменную, а если оптимизатора с таким названием в стандартной библиотеке `Keras` не было найдено, возвращается дефолтный `Adam` с дефолтными же параметрами.

Кратко: в `__init__()` создаётся объект класса с параметрами для всей нейросети в целом и оптимизатора в частности. Параметры оптимизатора можно передавать в любом порядке, главное, **после параметра `optimizer_name`**.

<a id="section4"></a>

## 4. Создание и обучение модели

**`def fit(self, x_train, y_train):`**

1. Принимает данные
2. Создаёт модель на основании параметров, переданных в `__init__()`
3. Обучает эту модель на переданных данных
4. Возвращает обученную модель

Подробнее:

Создаём заготовку для модели (своеобразный шомпур, на который затем будем нанизывать слои нейронной сети)

In [ ]:
model = Sequential()

Узнаём размерность данных

In [ ]:
x_train_shape = int(x_train.shape[1])
y_train_shape = int(y_train.shape[1])

После выполняется проверка, совпадает ли заданное количество нейронов на последнем (выходном) слое нейросети с размерностью **`y_train`**. Если нет, то выводит сообщение об ошибке, а если всё в порядке, то начинает формировать нейросеть.

Сначала проходит по всем значениям количеств нейронов для каждого уровня. В случае первого слоя вместе с количеством нейронов и параметром регуляризатора явно указывается размерность входных данных (требование `Keras`). В случае всех остальных слоёв просто добавляются новые слои с заданным количеством нейронов и, после этого, слой с указанной функцией активации.

In [ ]:
for index, layer_size in enumerate(self.hidden_layer_sizes):
                if index == 0:
                    model.add(Dense(layer_size, 
                                    input_dim=x_train_shape,
                                    kernel_regularizer=regularizers.l2(self.alpha)))
                    model.add(Activation(self.activations[index]))
                else:
                    model.add(Dense(layer_size,
                                    kernel_regularizer=regularizers.l2(self.alpha)))
                    model.add(Activation(self.activations[index]))

Потом происходит выбор оптимизатора. В функцию передаётся весь словарь параметров класса, а не только часть с параметрами, относящимися непосредственно к оптимизатору. Был проведен тест скорости времени выполнения обоих вариантов (передача всего словаря и передача переменной, содержащей только пары ключ-значение, относящиеся к оптимизатору). Итог – передача всего словаря параметров класса быстрее на 2 миллисекунды, вероятно, из-за отсутствия необходимости создавать дополнительную переменную.

In [ ]:
chosen_optimizer = None 
chosen_optimizer = get_optimizer(self.optimizer_name, self.__dict__)

Далее выполняется компиляция модели:

In [ ]:
model.compile(loss=self.loss_function, 
              optimizer=chosen_optimizer,
              metrics=self.metrics)

Выводится информация о готовой к работе модели:

In [ ]:
model.summary()

Добавляются колл-бэки и щепотка обратной совместимости:

In [ ]:
# Так называемые колл-бэки Keras принимает только в виде массива
used_callbacks = []

# На тот случай, если вдруг понадобится поддержка EarlyStopping,
# код честно нашел где-то на гитхабе в обсуждениях.
if self.early_stopping == True:
    early_stopping_callback = EarlyStopping(monitor="value_loss")
    used_callbacks.append(early_stopping_callback)

# Нужно для большей совместимости с прежним кодом
if self.batch_size == "auto":
    self.batch_size = 200

Наконец, происходит обучение модели:

In [ ]:
model.fit(x_train, 
          y_train,
          batch_size = self.batch_size,
          epochs = self.max_iter,
          verbose = self.verbose,
          callbacks = used_callbacks,
          shuffle = self.shuffle)

И возвращается готовая модель:

In [ ]:
return model

<a id="section5"></a>

## 5. Частичное обучение

В какой-то мере метод **`partial_fit()`** представляет в данном случае простую надстройку над уже описанной выше **`fit()`**. По сути, логика его работы сводится к тому, чтобы проверить, существует ли на диске в одной папке с исполняемым кодом файл сохранённой нейросети. Если есть, он открывает его и проводит обучение по вновь переданным данным. Если нет, он создает его, обучает и сохраняет вновь полученную модель в отдельный файл в той же директории. 

Две проблемы:
1. Название образующегося файла захардкожено. Решение: можно добавить его как параметр в функцию `partial_fit`.
2. **Более важно**: возможно, необходимо в `partial_fit` передавать все те же параметры, что и в обычный `fit`. Потому что вот сейчас я перечитал код и подумал, что `fit`-то выполняется в `partial_fit()`, но с какими параметрами??? Решение: передавать в `partial_fit()` те же параметры, что и в обычный `fit`.

In [ ]:
def partial_fit(self, x_train, y_train):
    	# метод для обучения сети в несколько заходов
    	# если файл с обученной моделью уже существует,
    	keras_model_filename="trained_keras_model.h5"
    	if os.path.isfile(keras_model_filename):
    		# загрузить его
    		model = load_model(keras_model_filename)
    		# дообучить модель
    		model.fit(x_train, y_train)
    		# сохранить модель
    		model.save(keras_model_filename)
    	else:
    		model = self.fit(x_train, y_train)
    		model.save(keras_model_filename)

**TODOs**:
- **(+)** переименовать max_iter в epochs
- **(+)** переименовать hidden_layer_sizes просто в layer_sizes
- **(+)** разобраться со входным слоем:
  - сделать число нейронов на первом слое таким же, как количество чисел в переданном массиве
  - почитать подробнее, для чего там нужен `input_dim` всё-таки
- **(+)** автоматическое выставление количества нейронов на выходном слое в зависимости от входных данных
- Обновить этот ноутбук под новую версию класса, включающую дропаут